# 1. Setup & Bus Data Load

Import required modules and packages.

In [ ]:
# import os so that environment variables can be accessed (for database password, etc.)
import os

# import time for timing various tasks
import time

# import math for mathematical functions
import math

# import pandas and numpy for data analysis
import pandas as pd
import numpy as np

# import transform_segments function for transforming data into segment format
from transform_data import transform_segments

# import convert_timestamp for various timestamp conversion functions
import convert_timestamp

# import matplotlib & seaborn for plotting and visualisaion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# import from sklearn for machine learning
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

# import pickle so that models can be saved to file
import pickle

Set the max number of columns & rows to display.

In [ ]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 500)

Load the bus data from file.

In [ ]:
start = time.time()
df = pd.read_csv('/data_analytics/data/segment_group_6.csv', sep=";", na_values=['\\N'])
end = time.time()
print(end - start)

Perform a check to see how many rows and columns are in the file.

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("Before any data cleaning, the CSV file contains", rows, "rows and", cols, "columns.")
print()

# 2. Initial Checks on the Bus Data

## 2.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

There are no duplicate rows or columns in the bus data.

## 2.2 Check for Null Features

In [ ]:
df.describe().T

There are no null features in the bus data.

## 2.3 Assign Features as Continuous or Categorical

Assign categorical and continous features, and update the type of all categorical features to 'category'.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'progrnumber', 'stoppointid']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 2.4 Check for Constant Categorical Features

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

There are no constant features to be droppped.

## 2.5 Check for Constant Continuous Features

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no constant features to be droppped.

# 3. Initial Checks for Missing Data

## 3.1 Categorical Features

Print details for the categorical columns:

In [ ]:
df[categorical_columns].describe().T

There is a full count for all categorical features.

## 3.2 Continuous Features

Print details for the continuous columns:

In [ ]:
df[continuous_columns].describe().T

There is a full count for all continuous features.

# 4. Transform the Bus Data

## 4.1 Initial Transformation

Bus data must be transformed so that each row of data holds information on one journey segment. This is done by calling the *transform_data* function:

In [ ]:
start = time.time()
df = transform_segments(df)
end = time.time()
print(end - start)

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After transformation, the dataframe contains", rows, "rows and", cols, "columns.")
print()

## 4.2 Check for Missing Data

First re-assign the transformed data as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

Then check for missing data:

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no features with missing data.

## 4.3 Add a Segment Feature

The new feature is added:

In [ ]:
# based on https://stackoverflow.com/questions/11858472/string-concatenation-of-two-pandas-columns
df = df.assign(segment=(df['stoppointid_first'].astype(int)).astype(str).str.cat((df['stoppointid_next'].astype(int)).astype(str), sep='_'))

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next', 'segment']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

## 4.4 Drop Segments

We then must drop data that isn't related to the segments that aren't in the current group:

In [ ]:
# define our list of valid segments for this group
valid_segments = ["2073_2074","3695_7115","3619_6087","2989_2990","1426_1427","545_546","907_908","754_755","1870_1871","174_175","3566_3567","2917_2918","1019_1020","7011_2171","834_835","83_84","6137_4256","6145_6146","730_5160","1167_1168","1442_7414","7446_7070","1724_1725","3321_2066","6021_7507","1224_1225","2102_2726","2688_2689","6341_6277","1757_1794","7227_6172","41_42","222_223","4571_2013","1311_1312","1313_1055","2016_2015","7070_7445","636_637","771_772","2372_2373","69_828","1005_1788","7057_4847","1096_1101","3497_3498","3508_3510","3054_3055","2568_2569","250_251","418_419","830_831","1262_4692","4002_5112","1302_1303","1441_1442","2648_2649","1819_1822","913_914","848_2795","4005_4006","7616_5081","1762_1763","1141_1142","3598_4465","3594_3595","102_6239","154_155","802_1654","1428_1429","1166_1167","3400_3401","149_150","3689_3690","3258_2060","3145_3146","141_142","1731_1732","7173_3811","1729_1730","1227_1253","3723_3724","4273_4274","2394_2395","1352_1353","2981_2991","3852_3853","2599_4577","752_753","1377_1378","1829_4997","1462_5149","2485_2486","4266_4267","1774_1775","6229_6364","3219_3220","3679_5075","3627_3628","2650_2652","588_589","1713_1714","4501_4935","1637_1638","3355_3356","1424_1425","6282_6335","2810_7387","4568_4569","651_4472","2070_2084","2680_2676","1085_1086","7366_7367","2066_2067","133_134","1229_1231","1949_1950","4917_4918","4246_4248","3597_3598","4623_4619","417_418","86_87","2536_2616","7672_826","1864_1865","1865_1866","445_4779","3619_3620","1062_1063","2634_2635","1314_1316","3798_3799","598_599","424_425","2660_2661","2082_2083","7039_7204","7170_3740","1622_1623","2137_2138","4533_4844","1507_1508","489_490","1022_1023","2559_2560","3898_3899","5013_5014","2495_2496","750_751","881_882","2399_2400","7031_4897","159_1539","1065_1066","1969_1970","3959_3960","36_37","117_118","22_23","4565_4566","1818_1819","1016_1017","7080_7081","726_727","3782_3783","1671_1672","3136_3138","1045_4710","7127_7143","1538_4543","749_750","3819_3820","119_44","3856_3857","490_491","1529_4542","7473_2679","1709_1528","3475_3476","1557_1181","4261_7271","2060_2061","2752_2753","3600_3601","2610_2554","3887_3888","4707_4708","1412_1413","1907_1908","3216_3217","4905_4906","2924_2925","4250_4251","193_138","3476_3477","2427_2428","4588_4589","2642_2643","3952_2190","3226_3227","1996_1997","3245_3212","3504_3505","2353_2357","3427_3428","2008_2009","2393_2394","1417_1418","4086_4087","4560_5126","211_212","1871_1872","919_7551","189_190","201_148","4226_4227","2261_4469","6130_1262","6343_4759","531_709","1463_4399","423_424","3923_3112","387_388","115_37","581_582","1200_4385","1259_1260","2910_2911","3352_3335","4253_4254","1081_2914","2189_1436","4091_6097","629_630","3236_3237","4287_4288","3928_3929","4231_7352","4620_7563","3892_3893","4211_4212"]

In [ ]:
# remove all other segments
df = df.loc[df['segment'].isin(valid_segments)]

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After dropping segments, the dataframe contains", rows, "rows and", cols, "columns.")
print()

## 4.5 Export the Data

The transformed data is exported at this point so that it can be reloaded easily without having to re-run the whole notebook:

In [ ]:
df.to_hdf('/data_analytics/data/segment_group_6_transformed.hdf', key='transformed', format='table')

Import the data when required:

In [ ]:
df = pd.read_hdf('/data_analytics/data/segment_group_6_transformed.hdf')

# 5. Import Weather Data

Weather data is loaded from a csv:

In [ ]:
df_weather = pd.read_csv('/data_analytics/data/weather.csv', sep=",", na_values=['\\N'])

## 5.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df_weather.duplicated()[df_weather.duplicated() == True].shape[0])
print('Duplicate columns:',df_weather.columns.size - df_weather.columns.unique().size)

There are no duplicate rows or columns so nothing needs to be dropped here.

## 5.2 Assign Features as Continuous or Categorical

Assign categorical and continuous features:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'irain', 'itemp', 'iwb']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_weather.select_dtypes(['float64', 'int64']).columns

## 5.3 Check for Missing Data, Constant Features, etc.

In [ ]:
# Print details for the categorical columns
df_weather[categorical_columns].describe().T

**itemp** and **iwb** are constant columns so can be dropped.

In [ ]:
# Print details for the categorical columns
df_weather[continuous_columns].describe().T

Investigate rows with missing data for rain:

In [ ]:
# select all rows where irain is not 0
df_weather.loc[df_weather['irain'] != 0]

There are only two rows where irain is not zero, these rows correspond to missing values for rain.

In [ ]:
# select other rows around the missing values
df_weather[6220:6240]

Given that there is no rain for the rest of the day, and given the high (for Ireland) temperature on the day, I think it's safe to replace the missing rain values with 0.

I will then drop the feature **irain** as it provides no useful information.

## 5.4 Replace Missing Weather Data

In [ ]:
# replace rain with 0 where irain is not 0
df_weather = df_weather.loc[df_weather['irain'] != -1]

In [ ]:
# check that values are updated
df_weather.loc[df_weather['irain'] != 0]

## 5.5 Drop Constant Weather Features

In [ ]:
df_weather = df_weather.drop(columns=['irain', 'itemp', 'iwb'])

# 6. Combine Bus and Weather Data

## 6.1 Split Timestamp for Weather Data

To merge the data, timestamps must be split into month, day and hour.

New features are added:

In [ ]:
df_weather['month'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_month_weather(x))

In [ ]:
df_weather['day'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_day_weather(x))

In [ ]:
df_weather['hour'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_hour_weather(x))

New features are updated to be categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'month', 'day', 'hour']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

## 6.2 Split Timestamp for Bus Data

New features are added:

In [ ]:
df['month'] = df.apply (lambda row: convert_timestamp.timestamp_to_month_bus(row['dayofservice'], \
                                                                                   row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df['day'] = df.apply (lambda row: convert_timestamp.timestamp_to_day_bus(row['dayofservice'], \
                                                                               row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df['hour'] = df['actualtime_arr_stop_first'].map(lambda x: convert_timestamp.timestamp_to_hour_bus(x))

## 6.3 Merge the Dataframes

In [ ]:
df = pd.merge(df, df_weather,  how='left', left_on=['month','day', 'hour'],\
                     right_on = ['month','day', 'hour'])

Check that there are no rows missing weather data:

In [ ]:
df[df.rain.isnull()]

Drop rows that are missing weather data:

In [ ]:
df = df[~df.rain.isnull()]

Update the data types for the new dataframe:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next', 'segment', \
                          'month', 'day', 'hour', 'record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

# 7. Create New Features

## 7.1 Day of Week Feature

The new feature is added:

In [ ]:
df['day_of_week'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_day_of_week(x))

## 7.2 Weekend/Weekday Feature

The new feature is added:

In [ ]:
df['weekday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_weekday_weekend(x))

## 7.3 Bank Holiday Feature

Make list of bank holidays for 2018 (based on https://www.officeholidays.com/countries/ireland/2018):

In [ ]:
holidays = ['2018-01-01 00:00:00', '2018-03-19 00:00:00', '2018-04-02 00:00:00', '2018-05-07 00:00:00', '2018-06-04 00:00:00',\
           '2018-08-06 00:00:00', '2018-10-29 00:00:00', '2018-12-25 00:00:00', '2018-12-26 00:00:00']

The new feature is added:

In [ ]:
df['bank_holiday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_bank_holiday(x, holidays))

## 7.4 Time Difference Feature

The new feature is added:

In [ ]:
df['time_diff'] = df['actualtime_arr_stop_next'] - df['actualtime_arr_stop_first']

Check for rows where the time difference is less than or equal to zero:

In [ ]:
df.loc[df['time_diff'] <= 0]

The rows that fail this test will be dropped:

In [ ]:
df = df.loc[df['time_diff'] > 0]

## 7.5 Update Data Types for the Features

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date', 'day_of_week', 'weekday', 'bank_holiday', 'segment']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 7.6 Drop Features

We will drop any features that we no longer need at this stage:

- dayofservice
- tripid
- direction
- progrnumber_first
- progrnumber_next
- record_date
- wetb
- dewpt
- vappr

In [ ]:
df = df.drop(columns=['dayofservice', 'tripid', 'direction', 'progrnumber_first', 'progrnumber_next', 'record_date', \
                     'wetb', 'dewpt', 'vappr'])

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['lineid', 'routeid', 'stoppointid_first', 'stoppointid_next',\
                          'month', 'day', 'hour', 'day_of_week', 'weekday', 'bank_holiday', 'segment']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 7.7 Export the Cleaned Data

The cleaned data is exported at this point so that it can be reloaded easily without having to re-run the whole notebook:

In [ ]:
df.to_hdf('/data_analytics/data/segment_group_6_cleaned.hdf', key='cleaned', format='table')

Import the data when required:

In [ ]:
df = pd.read_hdf('/data_analytics/data/segment_group_6_cleaned.hdf')

# 8. Plot Features

## 8.1 Individual Features

### 8.1.1 Histograms for the Continuous Features

In [ ]:
df[continuous_columns].hist(figsize=(30,30))

### 8.1.2 Box Plots for the Continuous Features

In [ ]:
for col in continuous_columns:
    f = df[col].plot(kind='box', figsize=(10,5))
    plt.show()

### 8.1.3 Bar Plots for the Categorical Features

In [ ]:
# select the features to plot
# some categorical features will not be plotted here due to the high number of values e.g. bus stop no.
categorical_columns_plot = df[['month', 'day', 'hour', 'day_of_week', 'weekday', 'bank_holiday']].columns

In [ ]:
for column in categorical_columns_plot:
    f = df[column].value_counts().plot(kind='bar', title=column, figsize=(8,6))
    plt.show()

## 8.2 Correlations between Features

### 8.2.1 Correlation Matrix for Continuous Features

In [ ]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
sns.set(style="white")

# Calculate correlation of all pairs of continuous features
corr = df[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)

We are mainly interested here in the correlation between the weather data and *time_diff*. The correlation for these features is quite low.

Some of the weather features have very high correlation with each other e.g. temp with wetb and vappr with dewpt. The weather features we can use will be dependent on what is available in the weather forecast we receive. However, if all features are available in the forecast we may need to remove some of the features due to high correlation.

### 8.2.2 Correlations between the Continuous Features & the Target Feature

Our target feature is:

- time_diff

I will plot scatter plots for each of the following combinations:

- actualtime_arr_stop_first vs. time_diff
- rain vs. time_diff
- temp vs. time_diff
- rhum vs. time_diff
- msl vs. time_diff

In [ ]:
df.plot.scatter(x='actualtime_arr_stop_first', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='rain', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='temp', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='rhum', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='msl', y='time_diff', c='purple')

### 8.2.3 Correlations between the Categorical Features & the Target Feature

Our target feature is:

- time_diff


In [ ]:
for col in categorical_columns_plot:
    df.boxplot(column=['time_diff'], by=[col], figsize=(8,5))

## 8.3 Plots by Segment

I will repeat some of the above graphs breaking them down by segment. I will plot for time_diff as this seems to have a stronger correlation based on the above graphs (for categorical features at least).

In [ ]:
# list all segments
segments = ["2073_2074","3695_7115","3619_6087","2989_2990","1426_1427","545_546","907_908","754_755","1870_1871","174_175","3566_3567","2917_2918","1019_1020","7011_2171","834_835","83_84","6137_4256","6145_6146","730_5160","1167_1168","1442_7414","7446_7070","1724_1725","3321_2066","6021_7507","1224_1225","2102_2726","2688_2689","6341_6277","1757_1794","7227_6172","41_42","222_223","4571_2013","1311_1312","1313_1055","2016_2015","7070_7445","636_637","771_772","2372_2373","69_828","1005_1788","7057_4847","1096_1101","3497_3498","3508_3510","3054_3055","2568_2569","250_251","418_419","830_831","1262_4692","4002_5112","1302_1303","1441_1442","2648_2649","1819_1822","913_914","848_2795","4005_4006","7616_5081","1762_1763","1141_1142","3598_4465","3594_3595","102_6239","154_155","802_1654","1428_1429","1166_1167","3400_3401","149_150","3689_3690","3258_2060","3145_3146","141_142","1731_1732","7173_3811","1729_1730","1227_1253","3723_3724","4273_4274","2394_2395","1352_1353","2981_2991","3852_3853","2599_4577","752_753","1377_1378","1829_4997","1462_5149","2485_2486","4266_4267","1774_1775","6229_6364","3219_3220","3679_5075","3627_3628","2650_2652","588_589","1713_1714","4501_4935","1637_1638","3355_3356","1424_1425","6282_6335","2810_7387","4568_4569","651_4472","2070_2084","2680_2676","1085_1086","7366_7367","2066_2067","133_134","1229_1231","1949_1950","4917_4918","4246_4248","3597_3598","4623_4619","417_418","86_87","2536_2616","7672_826","1864_1865","1865_1866","445_4779","3619_3620","1062_1063","2634_2635","1314_1316","3798_3799","598_599","424_425","2660_2661","2082_2083","7039_7204","7170_3740","1622_1623","2137_2138","4533_4844","1507_1508","489_490","1022_1023","2559_2560","3898_3899","5013_5014","2495_2496","750_751","881_882","2399_2400","7031_4897","159_1539","1065_1066","1969_1970","3959_3960","36_37","117_118","22_23","4565_4566","1818_1819","1016_1017","7080_7081","726_727","3782_3783","1671_1672","3136_3138","1045_4710","7127_7143","1538_4543","749_750","3819_3820","119_44","3856_3857","490_491","1529_4542","7473_2679","1709_1528","3475_3476","1557_1181","4261_7271","2060_2061","2752_2753","3600_3601","2610_2554","3887_3888","4707_4708","1412_1413","1907_1908","3216_3217","4905_4906","2924_2925","4250_4251","193_138","3476_3477","2427_2428","4588_4589","2642_2643","3952_2190","3226_3227","1996_1997","3245_3212","3504_3505","2353_2357","3427_3428","2008_2009","2393_2394","1417_1418","4086_4087","4560_5126","211_212","1871_1872","919_7551","189_190","201_148","4226_4227","2261_4469","6130_1262","6343_4759","531_709","1463_4399","423_424","3923_3112","387_388","115_37","581_582","1200_4385","1259_1260","2910_2911","3352_3335","4253_4254","1081_2914","2189_1436","4091_6097","629_630","3236_3237","4287_4288","3928_3929","4231_7352","4620_7563","3892_3893","4211_4212"]

### 8.3.1 Continuous Feature Correlations with time_diff by Segment

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    x = df_temp['actualtime_arr_stop_first']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("actualtime_arr_stop_first")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    x = df_temp['rain']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("rain")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    x = df_temp['temp']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("temp")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    x = df_temp['rhum']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("rhum")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    x = df_temp['msl']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("msl")
    plt.ylabel("time_diff")
    plt.show()

### 8.3.2 Categorical Feature Correlations with time_diff by Segment

I am leaving out some categorical features with high cardinality e.g. dayofservice.

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['lineid'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['routeid'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['direction'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['month'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['day'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['hour'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['day_of_week'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['weekday'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == segment]
    df_temp.boxplot(column=['time_diff'], by=['bank_holiday'], figsize=(8,5))

# 9. Data Integrity Checks

## 9.1 Deal with Outliers

Make a list of all segments:

In [ ]:
segments = ["2073_2074","3695_7115","3619_6087","2989_2990","1426_1427","545_546","907_908","754_755","1870_1871","174_175","3566_3567","2917_2918","1019_1020","7011_2171","834_835","83_84","6137_4256","6145_6146","730_5160","1167_1168","1442_7414","7446_7070","1724_1725","3321_2066","6021_7507","1224_1225","2102_2726","2688_2689","6341_6277","1757_1794","7227_6172","41_42","222_223","4571_2013","1311_1312","1313_1055","2016_2015","7070_7445","636_637","771_772","2372_2373","69_828","1005_1788","7057_4847","1096_1101","3497_3498","3508_3510","3054_3055","2568_2569","250_251","418_419","830_831","1262_4692","4002_5112","1302_1303","1441_1442","2648_2649","1819_1822","913_914","848_2795","4005_4006","7616_5081","1762_1763","1141_1142","3598_4465","3594_3595","102_6239","154_155","802_1654","1428_1429","1166_1167","3400_3401","149_150","3689_3690","3258_2060","3145_3146","141_142","1731_1732","7173_3811","1729_1730","1227_1253","3723_3724","4273_4274","2394_2395","1352_1353","2981_2991","3852_3853","2599_4577","752_753","1377_1378","1829_4997","1462_5149","2485_2486","4266_4267","1774_1775","6229_6364","3219_3220","3679_5075","3627_3628","2650_2652","588_589","1713_1714","4501_4935","1637_1638","3355_3356","1424_1425","6282_6335","2810_7387","4568_4569","651_4472","2070_2084","2680_2676","1085_1086","7366_7367","2066_2067","133_134","1229_1231","1949_1950","4917_4918","4246_4248","3597_3598","4623_4619","417_418","86_87","2536_2616","7672_826","1864_1865","1865_1866","445_4779","3619_3620","1062_1063","2634_2635","1314_1316","3798_3799","598_599","424_425","2660_2661","2082_2083","7039_7204","7170_3740","1622_1623","2137_2138","4533_4844","1507_1508","489_490","1022_1023","2559_2560","3898_3899","5013_5014","2495_2496","750_751","881_882","2399_2400","7031_4897","159_1539","1065_1066","1969_1970","3959_3960","36_37","117_118","22_23","4565_4566","1818_1819","1016_1017","7080_7081","726_727","3782_3783","1671_1672","3136_3138","1045_4710","7127_7143","1538_4543","749_750","3819_3820","119_44","3856_3857","490_491","1529_4542","7473_2679","1709_1528","3475_3476","1557_1181","4261_7271","2060_2061","2752_2753","3600_3601","2610_2554","3887_3888","4707_4708","1412_1413","1907_1908","3216_3217","4905_4906","2924_2925","4250_4251","193_138","3476_3477","2427_2428","4588_4589","2642_2643","3952_2190","3226_3227","1996_1997","3245_3212","3504_3505","2353_2357","3427_3428","2008_2009","2393_2394","1417_1418","4086_4087","4560_5126","211_212","1871_1872","919_7551","189_190","201_148","4226_4227","2261_4469","6130_1262","6343_4759","531_709","1463_4399","423_424","3923_3112","387_388","115_37","581_582","1200_4385","1259_1260","2910_2911","3352_3335","4253_4254","1081_2914","2189_1436","4091_6097","629_630","3236_3237","4287_4288","3928_3929","4231_7352","4620_7563","3892_3893","4211_4212"]

Get all outliers i.e. rows where the value is more than three times the standard deviation for the segment.

In [ ]:
# based on https://stackoverflow.com/questions/45928046/identifying-statistical-outliers-with-pandas-groupby-and-individual-columns
stds = 3.0
z = df[['segment', 'time_diff']].groupby('segment').transform(lambda group: (group - group.mean()).div(group.std()))
outliers = z.abs() > stds

Print the outliers:

In [ ]:
df[outliers.any(axis=1)]

Remove the outliers:

In [ ]:
df_outliers = df[outliers.any(axis=1)]
df = pd.concat([df,df_outliers]).drop_duplicates(keep=False)

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing outliers, we have", rows, "rows and", cols, "columns.")
print()

## 9.2 Segment Checks

We need to check if we are missing data for any segments.

In [ ]:
# loop through each segment and check how many rows we have for it
# if there are no rows for a particular segment, then add it to the missing list
missing = []
for segment in segments:
    #print(str(segment[0]) + " to " + str(segment[1]) + ": ")
    df_temp = df.loc[df['segment'] == segment]
    #print(str(df_temp.shape[0]) + "\n")
    if df_temp.shape[0] == 0:
        missing.append(segment)

In [ ]:
len(missing)

In [ ]:
print("Missing Segments")
for seg in missing:
    print(seg)

These segments do not appear in the data, but exist in current Dublin Bus routes. As such we must deal with them in some way as predictions may be requested for them.

# 10. Feature Selection for the Model

For the initial model for all routes, I will use the following features:

- time_diff (the target feature)
- actualtime_arr_stop_first
- segment
- hour
- day_of_week
- month
- weekday
- bank_holiday
- rain
- temp
- rhum
- msl

The following features have not been used for this initial model:

- dayofservice: the relevant information from this feature has been split into other features e.g. month, day_of_week
- tripid: we will not be able to provide this information to the model
- lineid: could look at using in future but as it is a high cardinality categorical feature I will leave it out for now
- routeid: we may not be able to map this to the timetable information we have and if not won't be able to provide it to the model
- direction: could look at including in a future model, although doesn't seem to have any real correlation with the target feature
- progrnumber_first: probably shouldn't use as routes can change over time so progrnumber for a stop may not be the same as what the model was trained on
- progrnumber_next: as above
- day: could be included in a future model, but seems to have less correlation with other features compared to day_of_week
- record_date: the relevant information from this feature has been split into other features e.g. month, day_of_week
- wetb: not available from OpenWeather
- dewpt: not available from OpenWeather
- vappr: not available from OpenWeather
- actualtime_arr_stop_next: was used in the 15a model as the target feature, could look at using it again instead of time_diff but we can't use both at once
- stoppointid_first: segment is now used instead
- stoppointid_next: segment is now used instead
- month: leaving out for now as our data is just for August

# 11. Prepare Categorical Features

## 11.1 Binary Encoding

We will use binary encoding for some features to avoid wrongly assigning an order to the categorical features where one isn't really present. The following features will be binary encoded:

- hour
- day_of_week
- month

**weekday** and **bank_holiday** are already binary so don't need to be encoded.

In [ ]:
# encode values for hour
df_dummies = pd.get_dummies(df['hour'], prefix='hour')
df = pd.concat([df, df_dummies], axis =1)

In [ ]:
# encode values for day_of_week
df_dummies = pd.get_dummies(df['day_of_week'], prefix='day_of_week')
df = pd.concat([df, df_dummies], axis =1)

In [ ]:
# encode values for month
df_dummies = pd.get_dummies(df['month'], prefix='month')
df = pd.concat([df, df_dummies], axis =1)

## 11.2 Mean/Target Encoding

We will use mean/target encoding for the **time_diff** feature as it has such high cardinality.

In [ ]:
means = df.groupby('segment')['time_diff'].mean().round()

In [ ]:
df['segment_means'] = df['segment'].map(means)

# 12. Split Test & Training Data

We will use out of time sampling to split our test and training data.

First we ensure that the data is sorted by date and time:

In [ ]:
df = df.sort_values(by=['month', 'day', 'actualtime_arr_stop_first'])

Data is split between training and test data (shuffle is false as we are doing out of time sampling):

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, shuffle=False)

# 13. Prepare the Features

In [ ]:
# Prepare the descriptive & target features for the training data
X_train = df_train[['actualtime_arr_stop_first','segment_means','rain','temp','rhum','msl','weekday','bank_holiday','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4','day_of_week_5','day_of_week_6','hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23',\
                    'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]
y_train = df_train.time_diff

In [ ]:
# Prepare the descriptive & target features for the test data
X_test = df_test[['actualtime_arr_stop_first','segment_means','rain','temp','rhum','msl','weekday','bank_holiday','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4','day_of_week_5','day_of_week_6','hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23',\
                    'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]
y_test = df_test.time_diff

In [ ]:
# normalise the features for training
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
del df_train
del df_test
del df
del outliers
del z
del means

# 14. Linear Regression

## 14.1 Train the Model

In [ ]:
linreg = linear_model.LinearRegression().fit(X_train, y_train)

## 14.2 Test the Model

In [ ]:
# make predictions based on the training data
linreg_predicted = (linreg.predict(X_test))

In [ ]:
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, linreg_predicted))
print()
print("Root Mean Squared Error: ", math.sqrt(metrics.mean_squared_error(y_test, linreg_predicted)))
print()
print("R Squared:", metrics.r2_score(y_test, linreg_predicted))

# 15. Gradient Tree Boosting

## 15.1 Train the Model

In [ ]:
# specify the GTB parameters
gtb = GradientBoostingRegressor()

In [ ]:
# Fit model on the training data
start = time.time()
gtb_model = gtb.fit(X_train_scaled, y_train)
end = time.time()
print(end - start)

## 15.2 Test the Model

In [ ]:
# make predictions based on the training data
gtb_predicted = (gtb_model.predict(X_test_scaled))

In [ ]:
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, gtb_predicted))
print()
print("Root Mean Squared Error: ", math.sqrt(metrics.mean_squared_error(y_test, gtb_predicted)))
print()
print("R Squared:", metrics.r2_score(y_test, gtb_predicted))